In [8]:
using Pkg
# Pkg.update()
Pkg.instantiate()

using DevCommunities
using SimpleHypergraphs
using StatsBase

# g = build_hg("../../hgs/apl.hg")
# num_nodes = nhv(g[1])
# num_edges = nhe(g[1])
# println("Number of nodes: $num_nodes")
# println("Number of edges: $num_edges")

my_tags_so = ["rust" ,"elixir","clojure", "typescript" ,"julia", "python", "delphi", "go" ,"sql", "c#" ,"kotlin", "swift" ,"dart" ,"html", "solidity", "javascript", "f#", "bash" ,"lisp", "apl"]
# order by lexigraphical order
my_tags_so = sort(my_tags_so)

20-element Vector{String}:
 "apl"
 "bash"
 "c#"
 "clojure"
 "dart"
 "delphi"
 "elixir"
 "f#"
 "go"
 "html"
 "javascript"
 "julia"
 "kotlin"
 "lisp"
 "python"
 "rust"
 "solidity"
 "sql"
 "swift"
 "typescript"

In [10]:
function build_hg_randoms(path)

    nodes = Dict{Int,Int}()
    degree_nodes = Dict{Int,Int}()
    nodes_per_edge = Dict{Int,Vector{Int}}()

    print(nodes)
    hg = Hypergraph(0,0)
    for line in eachline(path)
        # split elemnts of the line by comma
        d = split(line, ",")
        vs = map(x -> parse(Int,(strip(replace(x, r"\[|\]" => "")))), d)
        for v in vs
            if !haskey(nodes, v)
                v_id = SimpleHypergraphs.add_vertex!(hg)
                push!(nodes, v=>v_id)
            end
        end
        # add he 
        vertices = Dict{Int, Bool}(nodes[v] => true for v in vs)

        # count degree of each node
        for v in vs
            if !haskey(degree_nodes, v)
                degree_nodes[v] = 0
            end
            degree_nodes[v] += 1
        end
        
        he_id = SimpleHypergraphs.add_hyperedge!(hg; vertices = vertices)
        push!(nodes_per_edge, he_id => vs)  
    end

    sum_degree_nodes = sum(values(degree_nodes))
    sum_degree_edges = sum(values(length(nodes_per_edge)))

    function prob_nodes(v)
        return degree_nodes[v]/sum_degree_nodes
    end

    function prob_edges(e)
        return length(nodes_per_edge[e])/sum_degree_edges
    end

    random_nodes_probs = Dict{Int,Float64}()
    random_edges_probs = Dict{Int,Float64}()

    for key in keys(nodes)
        pr = prob_nodes(key)
        random_nodes_probs[key] = pr
    end

    for key in keys(nodes_per_edge)
        pr = prob_edges(key)
        random_edges_probs[key] = pr
    end

    random_nodes = Dict{Int,Int}()
    random_nodes_per_edge = Dict{Int,Vector{Int}}()

    random_hg = Hypergraph(0,0)

    rnp = collect(keys(sort(random_nodes_probs)))
    rnp_weights = pweights(ProbabilityWeights(collect(values(sort(random_nodes_probs)))))
    rep = collect(keys(sort(random_edges_probs)))
    rep_weights = pweights(ProbabilityWeights(collect(values(sort(random_edges_probs)))))

    for i in 1:sum_degree_nodes
        v = sample(rnp, rnp_weights)
        e = sample(rep, rep_weights)

        if !haskey(random_nodes, v)
            v_id = SimpleHypergraphs.add_vertex!(random_hg)
            push!(random_nodes, v=>v_id)
        end

        if !haskey(random_nodes_per_edge, e)
            random_nodes_per_edge[e] = []
        end
        random_nodes_per_edge[e] = push!(random_nodes_per_edge[e], v)        
    end

    for key in keys(random_nodes_per_edge)
        vertices = Dict{Int, Bool}(random_nodes[v] => true for v in random_nodes_per_edge[key])
        he_id = SimpleHypergraphs.add_hyperedge!(random_hg; vertices = vertices)
    end

    return random_hg
end

build_hg_randoms (generic function with 1 method)

In [ ]:

rg = build_hg_randoms("../../hgs/apl.hg")
hg_save("../../randoms/randomPIPE", rg)

# open file
r1 = open("../../randoms/randomPIPE")
# substitue in each line "=true" with ","
run(`sed -i '' 's/=true/,/g' ../../randoms/randomPIPE`)
# remove first line
run(`sed -i '' '1d' ../../randoms/randomPIPE`)
# hash and rewriting id
run(`python3 rewrite.py ../../randoms/randomPIPE 1`)

run(`bash orb.sh`)


# println(nhv(random_hg))
# println(nhe(random_hg))
# hg_save("../../randoms/realRandom", random_hg)


In [ ]:
 # open file
 r1 = open("../../randoms/realRandom")
 # substitue in each line "=true" with ","
 run(`sed -i '' 's/=true/,/g' ../../randoms/realRandom`)
 # remove first line
 run(`sed -i '' '1d' ../../randoms/realRandom`)
 # hash and rewriting id
 run(`python3 rewrite.py ../../randoms/realRandom 1`)
 close(r1)

In [11]:
all_arrays = []
for i in 1:20
    # g = build_hg("../../hgs/apl.hg")
    
    println("building hg for tag: "*my_tags_so[i])
    g = build_hg("../../hgs/"*my_tags_so[i]*".hg")

    num_nodes = nhv(g[1])
    num_edges = nhe(g[1])
    println("Number of nodes: $num_nodes")
    println("Number of edges: $num_edges")
    flush(stdout)

    total_random_motif_count = [0 for i in 1:26]
    # generate 5 model and do average counting
    for j in range(1,5)
        rm_pa = build_hg_randoms("../../hgs/"*my_tags_so[i]*".hg")
        hg_save("../../randoms/randomPIPE", rm_pa)
        
        # open file
        r1 = open("../../randoms/randomPIPE")
        # substitue in each line "=true" with ","
        run(`sed -i '' 's/=true/,/g' ../../randoms/randomPIPE`)
        # remove first line
        run(`sed -i '' '1d' ../../randoms/randomPIPE`)
        # hash and rewriting id
        run(`python3 rewrite.py ../../randoms/randomPIPE 1`)
        
        run(`bash orb.sh`)
        
        # run(`bash /Users/ddevin/Documents/MoCHy/run_exact.sh "/Users/ddevin/Documents/vscode/DevCommunities/randoms/randomPIPE.hg"`)
        # run(`bash /Users/ddevin/Documents/MoCHy/run_exact.sh "/Users/ddevin/Documents/vscode/DevCommunities/hgs/apl.hg"`)
        close(r1)
        r2 = open("/Users/ddevin/Documents/vscode/DevCommunities/script_dump/dumpPIPE.txt")
        array = []
        for line in eachline(r2)
            if occursin("motif", line)
                if occursin("counting", line)
                    continue
                end
                # find integers in line after ":"
                motif = parse.(Int64, split(line, ":")[2:end])
                push!(array, motif[1])
            end
        end
        # array of any to array of integers
        array = convert(Array{Int64}, array)
        total_random_motif_count = [x + y for (x, y) in zip(array, total_random_motif_count)]
        close(r2)
        println("done with random model "*string(j))
        flush(stdout)
        # flush
    end

    total_random_motif_count = [x / 5 for x in total_random_motif_count]
    total_random_motif_count = ceil.(Int64, total_random_motif_count)
    println(total_random_motif_count)
    push!(all_arrays, total_random_motif_count)
end

println(all_arrays)
# write to file
open("/Users/ddevin/Documents/vscode/DevCommunities/results/all_motifs_counted.csv", "w") do io
    for i in 1:20
        # write(io, string(my_tags_so[i]))
        # write(io, ",")
        for j in 1:26
            if length(all_arrays[i]) == 0
                write(io, string(0))
                write(io, ",")
            else
                write(io, string(all_arrays[i][j]))
                write(io, ",")
            end
        end
        write(io, "\n")
    end
end





building hg for tag: apl
Number of nodes: 1341
Number of edges: 1136


Dict{Int64, Int64}()

done with random model 1


Dict{Int64, Int64}()

done with random model 2


...

Dict{Int64, Int64}()

done with random model 5


[255554133, 601623192, 119860, 450270, 901371, 3451510, 0, 82, 124, 1259, 240, 6568, 0, 0, 0, 0, 0, 0, 1239711, 10487437, 8290747, 75201800, 0, 421, 7657, 38600]
Any

[[153671, 272179, 947, 2808, 5329, 16939, 0, 8, 5, 86, 25, 261, 0, 0, 1, 2, 0, 0, 3252, 16199, 15320, 92331, 0, 18, 165, 419], [31844146, 85084623, 68422, 339600, 702140, 3763084, 17, 341, 732, 8925, 4094, 63253, 0, 9, 53, 313, 0, 0, 1443003, 13133531, 9859064, 109687666, 0, 4975, 91150, 503942], [651463655, 1498253742, 229633, 938897, 1702107, 7162751, 102, 151, 302, 3185, 752, 15874, 0, 0, 0, 0, 0, 0, 6769743, 48311079, 41094540, 333941166, 0, 2053, 38748, 168703], [141302, 318769, 2019, 7231, 15534, 57981, 3, 72, 86, 829, 401, 3451, 0, 3, 20, 75, 0, 0, 10445, 64685, 49858, 414869, 0, 148, 1883, 7138], [98816921, 274164044, 61129, 345523, 529654, 3001592, 0, 71, 333, 1637, 857, 12918, 0, 0, 0, 12, 0, 0, 893564, 8203451, 6678661, 68506737, 0, 741, 12165, 68120], [3814027, 5055938, 41855, 84532, 206902, 421658, 67, 316, 683, 3138, 1372, 8549, 0, 3, 23, 53, 0, 0, 130533, 581307, 516042, 2467106, 0, 752, 6430, 14821], [402116, 524979, 4442, 14524, 21637, 68101, 10, 55, 124, 794, 366, 257

BoundsError: BoundsError: attempt to access 0-element Vector{Int64} at index [1]

In [18]:
############## SINGLE RUN ##################

g = build_hg("../../hgs/"*"python"*".hg")

num_nodes = nhv(g[1])
num_edges = nhe(g[1])
println("Number of nodes: $num_nodes")
println("Number of edges: $num_edges")
flush(stdout)

total_random_motif_count = [0 for i in 1:26]
# generate 5 model and do average counting
for j in range(1,5)
    rm_pa = build_hg_randoms("../../hgs/"*"python"*".hg")
    hg_save("../../randoms/randomPIPE", rm_pa)
    
    # open file
    r1 = open("../../randoms/randomPIPE")
    # substitue in each line "=true" with ","
    run(`sed -i '' 's/=true/,/g' ../../randoms/randomPIPE`)
    # remove first line
    run(`sed -i '' '1d' ../../randoms/randomPIPE`)
    # hash and rewriting id
    run(`python3 rewrite.py ../../randoms/randomPIPE 1`)
    
    run(`bash orb.sh`)
    
    # run(`bash /Users/ddevin/Documents/MoCHy/run_exact.sh "/Users/ddevin/Documents/vscode/DevCommunities/randoms/randomPIPE.hg"`)
    # run(`bash /Users/ddevin/Documents/MoCHy/run_exact.sh "/Users/ddevin/Documents/vscode/DevCommunities/hgs/apl.hg"`)
    close(r1)
    r2 = open("/Users/ddevin/Documents/vscode/DevCommunities/script_dump/dumpPIPE.txt")
    array = []
    for line in eachline(r2)
        if occursin("motif", line)
            if occursin("counting", line)
                continue
            end
            # find integers in line after ":"
            motif = parse.(Int64, split(line, ":")[2:end])
            push!(array, motif[1])
        end
    end
    # array of any to array of integers
    array = convert(Array{Int64}, array)
    total_random_motif_count = [x + y for (x, y) in zip(array, total_random_motif_count)]
    close(r2)
    println("done with random model "*string(j))
    println(total_random_motif_count)
    flush(stdout)
    # flush
end

total_random_motif_count = [x / 5 for x in total_random_motif_count]
total_random_motif_count = ceil.(Int64, total_random_motif_count)
println(total_random_motif_count)
# push!(all_arrays, total_random_motif_count)

Number of nodes: 295974
Number of edges: 431319


Dict{Int64, Int64}()

run_approx_ver2_par.sh: line 2:  1618 Killed                  ./approx_ver2_par 10000 4 $1 > /Users/ddevin/Documents/vscode/DevCommunities/script_dump/dumpPIPE.txt


done with random model 1


Dict{Int64, Int64}()

...

Dict{Int64, Int64}()

run_approx_ver2_par.sh: line 2:  1657 Killed                  ./approx_ver2_par 10000 4 $1 > /Users/ddevin/Documents/vscode/DevCommunities/script_dump/dumpPIPE.txt


run_approx_ver2_par.sh: line 2:  1670 Killed                  ./approx_ver2_par 10000 4 $1 > /Users/ddevin/Documents/vscode/DevCommunities/script_dump/dumpPIPE.txt


done with random model 5


Int64[]


In [ ]:
# num_motifs = [1077,818707,23,7,5702,6152,25,1,131,27,10,4,0,0,0,0,5,1,8390,1397,41097,124693,9,582,145,53]
# num_motifs = [2070, 11469, 44796591, 58275821, 16910646, 34130003, 16, 93583226, 404737, 293960662, 70840, 175279493, 1, 4172338, 55998189, 362027234, 0, 0, 31323, 12423927, 14751, 27466910, 0, 152556, 716568, 7659269]

function cps(motifs, motifs_random)
    eps = 1 #fixed in the paper
    deltas = []
    sum_deltas = 0
    for i in 1:length(motifs)
        t = (motifs[i]-motifs_random[i])/(motifs[i]+motifs_random[i]+eps)
        push!(deltas, t)
        sum_deltas += t^2
    end
    cp = []
    for i in 1:length(motifs)
        push!(cp, deltas[i]/sqrt(sum_deltas))
    end
    return cp
end

# cp = cps(num_motifs, motifs_random)
# println(cp)

In [ ]:
using CSV, DataFrames
data = CSV.read("../../hgs/motifs.csv", DataFrame, header=false)
# for each column create an array of the values
w = open("../../results/cpprofilesAPL.csv", "w")

for i in 1:size(data, 2)
    col = data[:, i]
    col = convert(Array{Int64, 1}, col)
    cp = cps(col, num_motifs)
    new_line = join(cp, ",")
    
    write(w, new_line * "\n")
end
close(w)



In [ ]:
using Plots, LinearAlgebra
using DelimitedFiles
# read matrix from csv file
m = readdlm("../../results/cpprofiles1000.csv", ',', Float64)
h = heatmap(m)

In [ ]:
using CSV, DataFrames
data = CSV.read("../../hgs/motifs.csv", DataFrame, header=false)
dataRandom = CSV.File("../../results/all_motifs_counted.csv", DataFrame, header=false, transpose=true)
# for each column create an array of the values
# w = open("../../results/cpprofilesALL.csv", "w")

# match column of data with column of dataRandom
for i in 1:size(data, 2)
    col = data[:, i]
    col = convert(Array{Int64, 1}, col)
    random_motif = dataRandom[:, i]
    cp = cps(col, random_motif)
    new_line = join(cp, ",")
    
    # write(w, new_line * "\n")
end

# for i in 1:size(data, 2)
#     col = data[:, i]
#     col = convert(Array{Int64, 1}, col)
#     random_motif = dataRandom[:, i]
#     cp = cps(col, num_motifs)
#     new_line = join(cp, ",")
    
#     # write(w, new_line * "\n")
# end
# # close(w)



In [ ]:
ccs = SimpleHypergraphs.get_connected_components(g[1])
ccs = sort(ccs, by=length, rev=true)
# distribution of component sizes
ccs_len = length.(ccs)



In [ ]:
using PyPlot
dict_degree = DevCommunities.degree_histogram(g[1], normalized=true)
# bar plot of the degree distribution
fig,axs = subplots()
axs.bar(collect(keys(dict_degree)), collect(values(dict_degree)), color="red")
gcf()
